In [1]:
import pandas as pd
from pandas import option_context
from sklearn.utils import class_weight
import numpy as np

import nltk
from nltk.tokenize import word_tokenize
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords

import re
import string

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer 
from sklearn.decomposition import NMF

from nltk.corpus import stopwords
from nltk.tokenize import MWETokenizer

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer 

import xgboost as xgb
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_absolute_error

from numpy import mean
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import GridSearchCV
from xgboost import XGBClassifier

In [4]:
stop = stopwords.words('english')
stop.extend(['hi', 'hey', 'hello','ha', 'followed','wa','dm','dont','cant','wont','get','still','like','need',
            'someone','people','im','ive','month','week','day','could','give','want','please','pls','since','one',
             'back','thanks','thank','take','doesnt','does', 'might', 'must','lock','access','sent','closed','use',
            'u','contact','new','email','time','message','tried','answer','waiting','reason','going',
            'work','agent','bank','card','trying','even','every'])

In [5]:
# # Extend for most common English adverbs

stop.extend(['up','so','out','just','now','how','then','more','also','here',
            'well','only','very','even','back','there','down','still','in',
            'as','to','when','never','really','most','on','why','about','over',
            'again','where','right','off','always','today','all','far','long',
            'away','yet','often','ever','however','almost','later','much',
            'once','least','ago','together','around','already','enough','both',
            'maybe','actually','probably','home','of course','perhaps','little',
            'else','sometimes','finally','less','better','early','especially',
            'either','quite','simply','nearly','soon','certainly','quickly',
            'no','recently','before','usually','thus','exactly','hard',
            'particularly','pretty','forward','ok','okay','clearly','indeed',
            'rather','that','tonight','close','suddenly','best','instead',
            'ahead','fast','alone','eventually','directly'])

# # Extend for most common irregular verbs (except pay,lose,send,buy,spend)

stop.extend(['say','make','go','take','come','see','know','get','got','give',
            'find','think','tell','become','show','leave','feel','put','bring',
            'begin','keep','hold','write','stand','hear','let','mean','set','meet',
            'run','sit','speak','lie','lead','read','grow','fall',
            'build','understand','draw','break','cut','rise','drive','wear',
            'choose'])

# ## Extend for prepositions
stop.extend(['without','among'])

## Load data

In [6]:
df_intent = pd.read_csv('training_base copy.csv')
df_intent.dropna(inplace=True)

In [7]:
X_tr = df_intent['tweet']
X_tr.shape

(3388,)

In [8]:
y_tr = df_intent[['intent']]

In [10]:
def remove_space(text):
    text = text.strip()
    text = re.sub("\s\s+", " ", text)
    return text

In [11]:
df_test = pd.read_csv('test_preds8_xgb_preds.csv')
df_test['tweet_compound'] = df_test.tweet_compound.map(remove_space)
df_test['tweet'] = df_test.tweet.map(remove_space)
df_test.shape

(200, 13)

In [18]:
y_test = df_test[['intent']]

## Baseline

## With XGBoost

In [22]:
# creating bag of words model 
cv = CountVectorizer(stop_words = stop,
                      max_features=880) 
  
X = cv.fit_transform(X_tr).toarray() 
y = y_tr.iloc[:, 0].values 

# fitting XGB to the training set 
gbm = xgb.XGBClassifier()

/Users/tawneykirkland/opt/anaconda3/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:383: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['course'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '


In [23]:
gbm.fit(X,y)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=0, num_parallel_tree=1,
              objective='multi:softprob', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=None, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [24]:
df_test_t = df_test['tweet']
X_t = cv.fit_transform(df_test_t).toarray() 
y_pred = gbm.predict(X_t)
gbm_pred = [round(value) for value in y_pred]

In [25]:
accuracy = accuracy_score(df_test['intent'], gbm_pred)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 32.00%


In [26]:
df_test['gbm_pred'] =  gbm_pred
df_test.groupby('gbm_pred').count()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,username,tweet,tweet_compound,intent,topic,neg,neu,pos,compound
gbm_pred,,,,,,,,,,,,
0,77,77,77,77,77,77,77,77,77,77,77,77
2,66,66,66,66,66,66,66,66,66,66,66,66
3,12,12,12,12,12,12,12,12,12,12,12,12
4,39,39,39,39,39,39,39,39,39,39,39,39
5,6,6,6,6,6,6,6,6,6,6,6,6


### Gridsearch

In [39]:
dtrain = xgb.DMatrix(X, label=y)
dtest = xgb.DMatrix(X_t, label=df_test['intent'])

In [40]:
params = {
    # Parameters that I will tune; currently set to default
    'max_depth':6,
    'min_child_weight': 1,
    'eta':.3,
    'subsample': 1,
    'colsample_bytree': 1,
    # Other parameters
    'objective':'multi:softmax',
    'num_class' : 6
}

In [43]:
params['eval_metric'] = "merror" # #(wrong cases)/#(all cases)
num_boost_round = 999

In [45]:
model = xgb.train(
    params,
    dtrain,
    num_boost_round=num_boost_round,
    evals=[(dtest, "Test")],
    early_stopping_rounds=10
)

print("Best merror: {:.2f} with {} rounds".format(
                 model.best_score,
                 model.best_iteration+1))

[0]	Test-merror:0.64500
Will train until Test-merror hasn't improved in 10 rounds.
[1]	Test-merror:0.64500
[2]	Test-merror:0.63000
[3]	Test-merror:0.62500
[4]	Test-merror:0.61500
[5]	Test-merror:0.63000
[6]	Test-merror:0.63500
[7]	Test-merror:0.63500
[8]	Test-merror:0.63500
[9]	Test-merror:0.63000
[10]	Test-merror:0.63000
[11]	Test-merror:0.62000
[12]	Test-merror:0.63000
[13]	Test-merror:0.65000
[14]	Test-merror:0.64000
Stopping. Best iteration:
[4]	Test-merror:0.61500

Best merror: 0.61 with 5 rounds


#### With cross validation

In [46]:
cv_results = xgb.cv(
    params,
    dtrain,
    num_boost_round=num_boost_round,
    seed=42,
    nfold=5,
    metrics={'merror'},
    early_stopping_rounds=10
)
cv_results

,train-merror-mean,train-merror-std,test-merror-mean,test-merror-std
0,0.443845,0.003623,0.482000,0.014395
1,0.427612,0.006441,0.473737,0.013811
2,0.419864,0.004645,0.471080,0.013087
3,0.410714,0.002408,0.465176,0.014068
4,0.402450,0.002259,0.464882,0.014084
...,...,...,...,...
85,0.216647,0.006546,0.412054,0.021328
86,0.215540,0.006222,0.411464,0.022861
87,0.214950,0.005885,0.411170,0.023478
88,0.213991,0.005754,0.411761,0.025834


In [47]:
cv_results['test-merror-mean'].min()

0.4108742

Best merror score across CV was 0.41 

### Tuning max_depth and min_child_weight

In [48]:
gridsearch_params = [
    (max_depth, min_child_weight)
    for max_depth in range(4,12)
    for min_child_weight in range(4,10)
]

In [49]:
# Define initial best params and MAE
min_merror = float("Inf")
best_params = None

for max_depth, min_child_weight in gridsearch_params:
    print("CV with max_depth={}, min_child_weight={}".format(
                             max_depth,
                             min_child_weight))
    # Update our parameters
    params['max_depth'] = max_depth
    params['min_child_weight'] = min_child_weight
    
    # Run CV
    cv_results = xgb.cv(
        params,
        dtrain,
        num_boost_round=num_boost_round,
        seed=42,
        nfold=5,
        metrics={'merror'},
        early_stopping_rounds=10
    )
    # Update best merror
    mean_merror = cv_results['test-merror-mean'].min()
    boost_rounds = cv_results['test-merror-mean'].argmin()
    print("\tMERROR {} for {} rounds".format(mean_merror, boost_rounds))
    if mean_merror < min_merror:
        min_merror = mean_merror
        best_params = (max_depth,min_child_weight)
print("Best params: {}, {}, MERROR: {}".format(best_params[0], best_params[1], min_merror))

CV with max_depth=4, min_child_weight=4
	MERROR 0.43271059999999995 for 55 rounds
CV with max_depth=4, min_child_weight=5
	MERROR 0.42887140000000007 for 72 rounds
CV with max_depth=4, min_child_weight=6
	MERROR 0.4344816000000001 for 68 rounds
CV with max_depth=4, min_child_weight=7
	MERROR 0.44304 for 51 rounds
CV with max_depth=4, min_child_weight=8
	MERROR 0.45514279999999996 for 19 rounds
CV with max_depth=4, min_child_weight=9
	MERROR 0.4510126 for 49 rounds
CV with max_depth=5, min_child_weight=4
	MERROR 0.42710480000000006 for 70 rounds
CV with max_depth=5, min_child_weight=5
	MERROR 0.42621760000000003 for 66 rounds
CV with max_depth=5, min_child_weight=6
	MERROR 0.4383176 for 38 rounds
CV with max_depth=5, min_child_weight=7
	MERROR 0.44540500000000005 for 23 rounds
CV with max_depth=5, min_child_weight=8
	MERROR 0.4433366 for 30 rounds
CV with max_depth=5, min_child_weight=9
	MERROR 0.4498316 for 26 rounds
CV with max_depth=6, min_child_weight=4
	MERROR 0.4235588 for 55 roun

In [54]:
params['max_depth'] = 11
params['min_child_weight'] = 4

## Tuning subsample and colsample_bytree

In [55]:
gridsearch_params2 = [
    (subsample, colsample)
    for subsample in [i/10. for i in range(7,11)]
    for colsample in [i/10. for i in range(7,11)]
]

In [56]:
min_merror = float("Inf")
best_params = None

# We start by the largest values and go down to the smallest
for subsample, colsample in reversed(gridsearch_params2):
    print("CV with subsample={}, colsample={}".format(
                             subsample,
                             colsample))
    
    # We update our parameters
    params['subsample'] = subsample
    params['colsample_bytree'] = colsample
    
    # Run CV
    cv_results = xgb.cv(
        params,
        dtrain,
        num_boost_round=num_boost_round,
        seed=42,
        nfold=5,
        metrics={'merror'},
        early_stopping_rounds=10
    )
    # Update best score
    mean_merror = cv_results['test-merror-mean'].min()
    boost_rounds = cv_results['test-merror-mean'].argmin()
    print("\tMERROR {} for {} rounds".format(mean_merror, boost_rounds))
    if mean_merror < min_merror:
        min_merror = mean_merror
        best_params = (subsample,colsample)
print("Best params: {}, {}, MERROR: {}".format(best_params[0], best_params[1], min_merror))


CV with subsample=1.0, colsample=1.0
	MERROR 0.4229678 for 44 rounds
CV with subsample=1.0, colsample=0.9
	MERROR 0.42444260000000006 for 27 rounds
CV with subsample=1.0, colsample=0.8
	MERROR 0.42474059999999997 for 34 rounds
CV with subsample=1.0, colsample=0.7
	MERROR 0.4291650000000001 for 29 rounds
CV with subsample=0.9, colsample=1.0
	MERROR 0.42385399999999995 for 29 rounds
CV with subsample=0.9, colsample=0.9
	MERROR 0.4253278 for 38 rounds
CV with subsample=0.9, colsample=0.8
	MERROR 0.4220804 for 37 rounds
CV with subsample=0.9, colsample=0.7
	MERROR 0.4268054 for 33 rounds
CV with subsample=0.8, colsample=1.0
	MERROR 0.425034 for 22 rounds
CV with subsample=0.8, colsample=0.9
	MERROR 0.42621460000000005 for 37 rounds
CV with subsample=0.8, colsample=0.8
	MERROR 0.4303446 for 23 rounds
CV with subsample=0.8, colsample=0.7
	MERROR 0.4312322 for 22 rounds
CV with subsample=0.7, colsample=1.0
	MERROR 0.42975900000000006 for 26 rounds
CV with subsample=0.7, colsample=0.9
	MERROR 

In [57]:
params['subsample'] = .9
params['colsample_bytree'] = 0.8

## Turning learning_rate

In [68]:
%time

min_merror = float("Inf")
best_params = None

for eta in [.3, .2, .1, .05, .01, .005]:
    print("CV with eta={}".format(eta))
    
    # We update our parameters
    params['eta'] = eta
    
    # Run CV
    cv_results = xgb.cv(
        params,
        dtrain,
        num_boost_round=num_boost_round,
        seed=42,
        nfold=5,
        metrics={'merror'},
        early_stopping_rounds=10
    )
    
    # Update best score
    mean_merror = cv_results['test-merror-mean'].min()
    boost_rounds = cv_results['test-merror-mean'].argmin()
    print("\tMERROR {} for {} rounds\n".format(mean_merror, boost_rounds))
    if mean_merror < min_merror:
        min_merror = mean_merror
        best_params = eta
print("Best params: {}, MERROR: {}".format(best_params, min_merror))

CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 6.91 µs
CV with eta=0.3
	MERROR 0.4220804 for 37 rounds

CV with eta=0.2
	MERROR 0.4306432 for 59 rounds

CV with eta=0.1
	MERROR 0.453078 for 3 rounds

CV with eta=0.05
	MERROR 0.45661779999999996 for 5 rounds

CV with eta=0.01
	MERROR 0.4589798 for 3 rounds

CV with eta=0.005
	MERROR 0.4616358 for 3 rounds

Best params: 0.3, MERROR: 0.4220804


In [69]:
params['eta'] = .3

In [70]:
params

{'max_depth': 11,
 'min_child_weight': 4,
 'eta': 0.3,
 'subsample': 0.9,
 'colsample_bytree': 0.8,
 'objective': 'multi:softmax',
 'num_class': 6,
 'eval_metric': 'merror'}

## Testing

In [71]:
model = xgb.train(
    params,
    dtrain,
    num_boost_round=num_boost_round,
    evals=[(dtest, "Test")],
    early_stopping_rounds=10
)

[0]	Test-merror:0.67500
Will train until Test-merror hasn't improved in 10 rounds.
[1]	Test-merror:0.65000
[2]	Test-merror:0.64000
[3]	Test-merror:0.64000
[4]	Test-merror:0.63500
[5]	Test-merror:0.63000
[6]	Test-merror:0.65000
[7]	Test-merror:0.65000
[8]	Test-merror:0.66500
[9]	Test-merror:0.67000
[10]	Test-merror:0.68000
[11]	Test-merror:0.66000
[12]	Test-merror:0.65500
[13]	Test-merror:0.66000
[14]	Test-merror:0.65000
[15]	Test-merror:0.65500
Stopping. Best iteration:
[5]	Test-merror:0.63000



In [72]:
num_boost_round = model.best_iteration + 1
best_model = xgb.train(
    params,
    dtrain,
    num_boost_round=num_boost_round,
    evals=[(dtest, "Test")]
)

[0]	Test-merror:0.67500
[1]	Test-merror:0.65000
[2]	Test-merror:0.64000
[3]	Test-merror:0.64000
[4]	Test-merror:0.63500
[5]	Test-merror:0.63000


In [73]:
best_model.save_model("my_model.model")

In [ ]:
# loaded_model = xgb.Booster()
# loaded_model.load_model("my_model.model")